In [ ]:
'''
大作业：四轴飞行器悬浮
【作业五 大作业评分标准】
（需要保留notebook上每个cell运行之后的log信息）：
1、完全没有log信息，59分
2、完成部分函数的编写，有部分log信息，但代码运行失败，70分
3、代码运行成功，但Test reward收敛效果不好（所有分数均低于6000），80分
4、代码运行成功，Test reward有上涨趋势，且最后的Evaluate reward在[6000, 8000]之间，90分
5、代码运行成功，Test reward有上涨趋势，且最后的Evaluate reward大于8000，100分
'''

In [6]:
import os
import numpy as np
import parl
from parl import layers
from paddle import fluid
from parl.utils import logger
from parl.utils import action_mapping 
from parl.utils import ReplayMemory 
from rlschool import make_env 
from parl.algorithms import DDPG

In [2]:
ACTOR_LR  = 0.0005        # Actor网络更新的 learning rate
CRITIC_LR = 0.005         # Critic网络更新的 learning rate
GAMMA = 0.9               # reward 的衰减因子，一般取 0.9 到 0.999 不等
TAU   = 0.001             # target_model 跟 model 同步参数 的 软更新参数
MEMORY_SIZE        = 1e4  # replay memory的大小，越大越占用内存
MEMORY_WARMUP_SIZE = 1e4  # replay_memory 里需要预存一些经验数据，再从里面sample一个batch的经验让agent去learn
REWARD_SCALE = 0.01       # reward 的缩放因子
BATCH_SIZE   = 256        # 每次给agent learn的数据数量，从replay memory随机里sample一批数据出来
TRAIN_TOTAL_STEPS = 1e6   # 总训练步数
TEST_EVERY_STEPS  = 1e4   # 每个N步评估一下算法效果，每次评估5个episode求平均reward

In [3]:
class ActorModel(parl.Model):
    def __init__(self, act_dim):
        self.fc1 = layers.fc(size=64, act='relu')
        self.fc2 = layers.fc(size=64, act='relu')
        self.fc3 = layers.fc(size=act_dim, act='tanh')

    def policy(self, obs):
        hidden1 = self.fc1(obs)
        hidden2 = self.fc2(hidden1)
        logits  = self.fc3(hidden2)
        return logits

In [4]:
class CriticModel(parl.Model):
    def __init__(self):
        hidden_size = 64
        self.fc1 = layers.fc(size=hidden_size, act='relu')
        self.fc2 = layers.fc(size=hidden_size, act='relu')
        self.fc3 = layers.fc(size=1, act=None)
    def value(self, obs, act):
        concat = layers.concat([obs, act], axis=1)
        hidden1 = self.fc1(concat)
        hidden2 = self.fc2(hidden1)
        Q = self.fc3(hidden2)
        Q = layers.squeeze(Q, axes=[1])
        return Q

In [5]:
class QuadrotorModel(parl.Model):
    def __init__(self, act_dim):
        self.actor_model = ActorModel(act_dim)
        self.critic_model = CriticModel()

    def policy(self, obs):
        return self.actor_model.policy(obs)

    def value(self, obs, act):
        return self.critic_model.value(obs, act)

    def get_actor_params(self):
        return self.actor_model.parameters()

In [7]:
class QuadrotorAgent(parl.Agent):
    def __init__(self, algorithm, obs_dim, act_dim=4):
        assert isinstance(obs_dim, int)
        assert isinstance(act_dim, int)
        self.obs_dim = obs_dim
        self.act_dim = act_dim
        super(QuadrotorAgent, self).__init__(algorithm)

        # 注意：最开始先同步self.model和self.target_model的参数.
        self.alg.sync_target(decay=0)

    def build_program(self):
        self.pred_program = fluid.Program()
        self.learn_program = fluid.Program()

        with fluid.program_guard(self.pred_program):
            obs = layers.data(
                name='obs', shape=[self.obs_dim], dtype='float32')
            self.pred_act = self.alg.predict(obs)

        with fluid.program_guard(self.learn_program):
            obs = layers.data(
                name='obs', shape=[self.obs_dim], dtype='float32')
            act = layers.data(
                name='act', shape=[self.act_dim], dtype='float32')
            reward = layers.data(name='reward', shape=[], dtype='float32')
            next_obs = layers.data(
                name='next_obs', shape=[self.obs_dim], dtype='float32')
            terminal = layers.data(name='terminal', shape=[], dtype='bool')
            _, self.critic_cost = self.alg.learn(obs, act, reward, next_obs,
                                                 terminal)

    def predict(self, obs):
        obs = np.expand_dims(obs, axis=0)
        act = self.fluid_executor.run(
            self.pred_program, feed={'obs': obs},
            fetch_list=[self.pred_act])[0]
        act = np.squeeze(act)
        return act

    def learn(self, obs, act, reward, next_obs, terminal):
        feed = {
            'obs': obs,
            'act': act,
            'reward': reward,
            'next_obs': next_obs,
            'terminal': terminal
        }
        critic_cost = self.fluid_executor.run(
            self.learn_program, feed=feed, fetch_list=[self.critic_cost])[0]
        self.alg.sync_target()
        return critic_cost

In [8]:
def run_episode(env, agent, rpm):
    obs = env.reset()
    total_reward, steps = 0, 0
    while True:
        steps += 1
        batch_obs = np.expand_dims(obs, axis=0)
        action = agent.predict(batch_obs.astype('float32'))
        action = np.squeeze(action)
        action = np.clip(np.random.normal(action, 1.0), -1.0, 1.0)
        action = action_mapping(action, env.action_space.low[0],
                                env.action_space.high[0])
        next_obs, reward, done, info = env.step(action)
        rpm.append(obs, action, REWARD_SCALE * reward, next_obs, done)

        if rpm.size() > MEMORY_WARMUP_SIZE:
            batch_obs, batch_action, batch_reward, batch_next_obs, \
                    batch_terminal = rpm.sample_batch(BATCH_SIZE)
            critic_cost = agent.learn(batch_obs, batch_action, batch_reward,
                                      batch_next_obs, batch_terminal)

        obs = next_obs
        total_reward += reward

        if done:
            break
    return total_reward, steps

# 评估 agent, 跑 5 个episode，总reward求平均
def evaluate(env, agent):
    eval_reward = []
    for i in range(5):
        obs = env.reset()
        total_reward, steps = 0, 0
        while True:
            batch_obs = np.expand_dims(obs, axis=0)
            action = agent.predict(batch_obs.astype('float32'))
            action = np.squeeze(action)
            action = np.clip(action, -1.0, 1.0)
            action = action_mapping(action, env.action_space.low[0], 
                                    env.action_space.high[0])
            next_obs, reward, done, info = env.step(action)

            obs = next_obs
            total_reward += reward
            steps += 1

            if done:
                break
        eval_reward.append(total_reward)
    return np.mean(eval_reward)

In [11]:
# 创建飞行器环境
env = make_env("Quadrotor", task="hovering_control")
env.reset()
obs_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]


# 根据parl框架构建agent
model = QuadrotorModel(act_dim)
algorithm = DDPG(model, gamma=GAMMA, tau=TAU, actor_lr=ACTOR_LR, critic_lr=CRITIC_LR)
agent = QuadrotorAgent(algorithm, obs_dim, act_dim)
agent.restore('./model_dir/Pre_Training.ckpt')

rpm = ReplayMemory(int(MEMORY_SIZE), obs_dim, act_dim)

[06-26 18:40:31 MainThread @machine_info.py:86] nvidia-smi -L found gpu count: 1
[06-26 18:40:31 MainThread @machine_info.py:86] nvidia-smi -L found gpu count: 1
[06-26 18:40:31 MainThread @machine_info.py:86] nvidia-smi -L found gpu count: 1


In [12]:
#启动训练
test_flag = 0
total_steps = 0
while total_steps < TRAIN_TOTAL_STEPS:
    train_reward, steps = run_episode(env, agent, rpm)
    total_steps += steps
    #logger.info('Steps: {} Reward: {}'.format(total_steps, train_reward)) # 打印训练reward
    if total_steps // TEST_EVERY_STEPS >= test_flag: # 每隔一定step数，评估一次模型
        while total_steps // TEST_EVERY_STEPS >= test_flag:
                test_flag += 1
        evaluate_reward = evaluate(env, agent)
        logger.info('Steps {}, Test reward: {}'.format(
                total_steps, evaluate_reward)) # 打印评估的reward
        # 每评估一次，就保存一次模型，以训练的step数命名
        ckpt = 'model_dir/steps_{}.ckpt'.format(total_steps)
        agent.save(ckpt)
        

[06-26 18:41:38 MainThread @<ipython-input-12-efce29af3bdd>:13] Steps 1000, Test reward: 8457.575368518274
[06-26 18:43:26 MainThread @<ipython-input-12-efce29af3bdd>:13] Steps 10000, Test reward: 8413.648278417646
[06-26 18:45:21 MainThread @<ipython-input-12-efce29af3bdd>:13] Steps 20000, Test reward: 8274.117524445575
[06-26 18:47:15 MainThread @<ipython-input-12-efce29af3bdd>:13] Steps 30000, Test reward: 8433.745273897543
[06-26 18:49:09 MainThread @<ipython-input-12-efce29af3bdd>:13] Steps 40000, Test reward: 8491.882887179145
[06-26 18:51:04 MainThread @<ipython-input-12-efce29af3bdd>:13] Steps 50000, Test reward: 8423.17295522265
[06-26 18:52:58 MainThread @<ipython-input-12-efce29af3bdd>:13] Steps 60000, Test reward: 8484.636218737905
[06-26 18:54:53 MainThread @<ipython-input-12-efce29af3bdd>:13] Steps 70000, Test reward: 8555.556190381061
[06-26 18:56:48 MainThread @<ipython-input-12-efce29af3bdd>:13] Steps 80000, Test reward: 8308.305733207244
[06-26 18:58:43 MainThread @<i

[06-26 20:56:07 MainThread @<ipython-input-12-efce29af3bdd>:13] Steps 700000, Test reward: 8302.525390305636
[06-26 20:58:02 MainThread @<ipython-input-12-efce29af3bdd>:13] Steps 710000, Test reward: 8535.029504433778
[06-26 20:59:58 MainThread @<ipython-input-12-efce29af3bdd>:13] Steps 720000, Test reward: 8343.93996737016
[06-26 21:01:49 MainThread @<ipython-input-12-efce29af3bdd>:13] Steps 730000, Test reward: 8176.534532100722
[06-26 21:03:37 MainThread @<ipython-input-12-efce29af3bdd>:13] Steps 740000, Test reward: 8242.895155016737
[06-26 21:05:23 MainThread @<ipython-input-12-efce29af3bdd>:13] Steps 750000, Test reward: 8350.851381578424
[06-26 21:07:10 MainThread @<ipython-input-12-efce29af3bdd>:13] Steps 760000, Test reward: 8329.807101654656
[06-26 21:08:56 MainThread @<ipython-input-12-efce29af3bdd>:13] Steps 770000, Test reward: 8358.315287197142
[06-26 21:10:43 MainThread @<ipython-input-12-efce29af3bdd>:13] Steps 780000, Test reward: 8085.545040758535
[06-26 21:12:30 Main

In [13]:
ckpt = 'model_dir/steps_1000000.ckpt'  # 请设置ckpt为你训练中效果最好的一次评估保存的模型文件名称

agent.restore(ckpt)
evaluate_reward = evaluate(env, agent)
logger.info('Evaluate reward: {}'.format(evaluate_reward)) # 打印评估的reward

[06-26 22:27:58 MainThread @<ipython-input-13-8ba810ab151b>:5] Evaluate reward: 8446.997822155343
